## Import

In [1]:
# for read data
import os

# for Data Preprocessing
import pandas as pd
import numpy as np

## Read Data

In [2]:
path = (os.path.abspath("./input"))

# Incomplete Data Cleansing
train = pd.read_csv(path + '/X_train_IDC.csv', encoding='cp949')
test = pd.read_csv(path + '/X_test_IDC.csv', encoding='cp949')
y_train = pd.read_csv(path + '/y_train.csv')

man, woman = pd.read_pickle(path+'/DC_sex_1.pkl')

In [3]:
# Concat train, test data
train_ID = train.custid.unique()
test_ID = test.custid.unique()

data = pd.concat([train, test]).reset_index(drop=True)
display(data)

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,...,net_amt,inst_mon,inst_fee,REAL_PURCHASE,DISCOUNT,DISCOUNT_PER,REFUND,REFUND_AMT,IMPORT_AMT,INSTFEE_AMT
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,...,81000,3,0,1,1,0.10,0,0,81000.0,0.0
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,...,35100,1,0,1,1,0.10,0,0,35100.0,0.0
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,...,157500,3,0,1,1,0.10,0,0,157500.0,0.0
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,...,409500,3,0,1,1,0.10,0,0,409500.0,0.0
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,...,90000,3,0,1,1,0.10,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040034,49993,1,31,수,1750,신촌점,4405551020474,톰키드,아동,아동,...,20000,1,0,1,0,0.00,0,0,0.0,0.0
1040035,49993,1,31,수,1833,신촌점,2139140008300,폴로화장품,향수,화장품,...,66500,1,0,1,1,0.05,0,0,0.0,0.0
1040036,49994,4,14,토,1750,본점,4230120011274,스테파넬,영트랜드,영트랜디,...,39000,1,0,1,0,0.00,0,0,0.0,0.0
1040037,49994,4,14,토,1810,본점,4409271026010,써스데이아일앤드,진캐주얼,진캐주얼,...,34200,1,0,1,0,0.00,0,0,0.0,0.0


### Column Generation
Feature Generation에 사용할 새로운 열을 생성하고자 한다. 원본 데이터와 헷갈리지 않게 생성하는 열의 이름은 대문자로 한다.

- **[SALES_DATE, SALES_DAY, SALES_SEC]**<br>
  매달 일수는 고정되어 있음으로 구매주기, 매장이용시간 등을 계산하기 위해 임의의 년도의 날짜 데이터로 생성한다. `연도는 활용해선 안된다.`<br>
  또한 편의를 위해 'YYYYmmdd' 날짜 데이터를 생성한다. 시간과 분을 분으로 표현한 데이터를 생성한다.

In [4]:
data['SALES_DATE'] = pd.to_datetime(data['sales_month'].apply(lambda x: '2018'+str(x).zfill(2) if x < 5 else '2017'+str(x).zfill(2))\
                                    + data.sales_day.astype(str).str.zfill(2)\
                                    + data.sales_time.astype(str), format='%Y%m%d%H%M')
data['SALES_DAY'] = data.SALES_DATE.dt.strftime('%Y%m%d')
# 3등 Feature 사용을 위해 만듦
data['SALES_MIN'] = data['sales_time'].apply(lambda x : x//100*60 + x%100)

# SALES_DATE가 담은 데이터 중 활용하지 않을 데이터는 제거한다.
del data['sales_day'], data['sales_time']

- **[NEXT_PURCHASE, DUPLICATE_VISIT]**<br>
  다음 상품을 구매하는 데까지 걸리는 분간격 데이터와 같은날 여러번 방문한 여부 데이터를 생성한다.<br>
  [남녀 쇼핑시간 차이] https://www.folin.co/story/524

In [5]:
time = data[['custid', 'SALES_DATE']].sort_values(by=['custid', 'SALES_DATE'])
time = time.diff()

In [6]:
time['minute'] = time.SALES_DATE.dt.total_seconds()//60
time['hour'] = time.minute//60
time['same_id'] = time.custid.apply(lambda x: 0 if x != 0 else 1)
time['same_day'] = time.SALES_DATE.dt.days.apply(lambda x: 1 if x == 0 else 0)
time['same_visit'] = time.hour.apply(lambda x: 1 if 0 <= x < 3 else 0)

In [7]:
data = data.sort_values(by=['custid', 'SALES_DATE'])
data['NEXT_PURCHASE'] = time['same_id'] * time['same_day'] * time['same_visit'] * time['minute']
data.NEXT_PURCHASE.fillna(0, inplace=True)   # 1행의 NaT를 0으로 채움

In [8]:
data['DUPLICATE_VISIT'] = pd.merge(data.reset_index()['index'],
                                   time.query('(same_id == 1) & (same_day == 1)').hour.apply(lambda x: 1 if x > 3 else 0).reset_index(),
                                   on='index', how='outer').fillna(0).iloc[:, -1]

- **[SALES_WEEKEND]**<br>
  주말에 구매한 데이터인지를 의미하는 데이터를 생성한다.

In [9]:
data['SALES_WEEKEND'] = data.sales_dayofweek.apply(lambda x: 1 if (x=="토" or x=="일") else 0)

- **[DAILY, LEAVE_OFFICE]**<br>
  회사원을 구분하기 위해 평일 중 퇴근시간이 아닐때 방문하였는지, 퇴근시간에 방문하였는지를 의미하는 데이터를 생성한다.

In [10]:
data['DAILY'] = data.SALES_WEEKEND.apply(lambda x: 1 if x==0 else 0)\
                * data.SALES_DATE.dt.hour.apply(lambda x: 1 if 9<=x<=18 else 0)
data['LEAVE_OFFICE'] = data.SALES_WEEKEND.apply(lambda x: 1 if x==0 else 0)\
                       * data.SALES_DATE.dt.hour.apply(lambda x: 0 if x>=18 else 1)

- **[WEIGHT_{group}, FAVOR_{group}]**<br>
  group별 상위 10개의 브랜드에 해당하는 상품을 구매한건지, 구매한 상품의 순위가중치를 의미하는 데이터를 생성한다.

In [11]:
weight = pd.merge(train, y_train, on='custid', how='inner')
weight_list = weight.groupby('group')['brd_nm'].agg(lambda x: x.value_counts().index.tolist()[1:11]).reset_index()

for i, g, b in weight_list.itertuples():
    data[f'WEIGHT_{g}'] = data.brd_nm.apply(lambda x: 10 - b.index(x) if x in b else 0)
    data[f'FAVOR_{g}'] = data.brd_nm.apply(lambda x: 1 if x in b else 0)
    
data['WEIGHT_SUM'] = data[[i for i in data.columns if 'WEIGHT' in i]].sum(axis=1)

- **[MAN, WOMAN]**<br>
  corner_nm, pc_nm, part_nm에 "남성", "여성"이 들어간 상품의 구매여부를 생성한다.

In [12]:
data['MAN'] = (data['brd_nm'].apply(lambda x: 1 if x in man['brd_nm'] else 0) +\
               data['corner_nm'].str.contains('남성').astype(int) +\
               data['corner_nm'].apply(lambda x: 1 if x in man['corner_nm'] else 0) +\
               data['pc_nm'].str.contains('남성').astype(int) +\
               data['pc_nm'].apply(lambda x: 1 if x in man['pc_nm'] else 0) +\
               data['part_nm'].str.contains('남성').astype(int))\
               .apply(lambda x: 0 if x == 0 else 1)

In [13]:
data['WOMAN'] = (data['corner_nm'].str.contains('여성').astype(int) +\
                 data['corner_nm'].apply(lambda x: 1 if x in woman['corner_nm'] else 0) +\
                 data['pc_nm'].str.contains('여성').astype(int) +\
                 data['pc_nm'].apply(lambda x: 1 if x in woman['pc_nm'] else 0) +\
                 data['part_nm'].str.contains('여성').astype(int))\
                 .apply(lambda x: 0 if x == 0 else 1)

- **[BABY, CHILD]**<br>
  corner_nm에 "유아" 또는 "신생아", "아동"이 들어간 상품의 구매여부를 생성한다.

In [14]:
data['BABY'] = data['corner_nm'].str.contains('유아').astype(int) + data['corner_nm'].str.contains('신생아').astype(int)
data['CHILD'] = data['corner_nm'].str.contains('아동').astype(int)

- **[COSMETIC]**<br>
  corner_nm에 "화장품"이 들어간 상품의 구매여부를 생성한다.

In [15]:
data['COSMETIC'] = data['corner_nm'].str.contains('화장품').astype(int)

- **[WITHOUT_INST]**<br>
  롯데백화점의 무이자 조건을 근거로 무이자 혜택 카드를 사용한 것인지를 의미하는 데이터를 생성한다.

In [16]:
data['WITHOUT_INST'] = pd.merge(data.reset_index()['index'], 
                                data.query('net_amt >= 50000 and inst_mon > 1')['inst_fee'].apply(lambda x : 1 if x == 0 else 0).reset_index(),
                                on='index', how='outer').iloc[:, -1].fillna(0)

### $~~~$ Confirm data

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040039 entries, 0 to 1040038
Data columns (total 55 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   custid           1040039 non-null  int64         
 1   sales_month      1040039 non-null  int64         
 2   sales_dayofweek  1040039 non-null  object        
 3   str_nm           1040039 non-null  object        
 4   goodcd           1040039 non-null  int64         
 5   brd_nm           1040039 non-null  object        
 6   corner_nm        1040039 non-null  object        
 7   pc_nm            1040039 non-null  object        
 8   part_nm          1040039 non-null  object        
 9   team_nm          1040039 non-null  object        
 10  buyer_nm         1040039 non-null  object        
 11  import_flg       1040039 non-null  int64         
 12  tot_amt          1040039 non-null  int64         
 13  dis_amt          1040039 non-null  int64         
 14  ne

### $~~~$ Split train, test data

In [18]:
train = data.query('custid in @train_ID').reset_index(drop=True)
test = data.query('custid in @test_ID').reset_index(drop=True)

### $~~~$ Save train, test data

In [19]:
train.to_csv(path +'/X_train_DC.csv', index=False, encoding='cp949')
test.to_csv(path +'/X_test_DC.csv', index=False, encoding='cp949')